<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Generate Readme for Awesome Notebooks

In [ ]:
import os
import naas_drivers
import urllib.parse

import copy
import nbformat
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)

## Variables

In [ ]:
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/jupyter-naas/awesome-notebooks/tree/master'
github_download_url = 'https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/'
naas_download_url='https://app.naas.ai/user-redirect/naas/downloader?url='

## Get files list

In [ ]:
total = []
for root, directories, files in os.walk(current_file, topdown=False):
    total.append({"root": root, "directories":directories, "files":files})
total.sort(key=lambda x: x.get('root'))

## Set 'Add on Naas' link on notebook

In [ ]:
def set_link_in_source(source, add_link, title_index, text):
    insert_index = len(source)
    if source.find('\n', title_index) != -1:
        insert_index = source.index('\n', title_index)
    if source.find(text) != -1:
        index = source.index(text)
        link_size = 0
        if source[index - 1] == '\n':
            link_size += 1
            index -= 1
        if source.find('(', title_index) != -1 and source.find(')', title_index) != -1:
           link_size += source.index(')', title_index) - source.index('(', title_index) + len(text) + 1
        source = source[:index] + source[index + link_size:]
    source = source[:insert_index] + f"\n{text}({add_link})" + source[insert_index:]
    return source

def set_notebook_link(notebook_path, add_link, text="Open in Naas"):
    count = 0
    nb = load_notebook_node(notebook_path)
    nb = copy.deepcopy(nb)
    text = '[<button class="lm-Widget p-Widget jupyter-widgets jupyter-button widget-button mod-primary">' + text + '</button>]'
    for cell in nb.cells:
        source = cell.source
        if source.find('#') != -1:
            index = source.index('#')
            while (index != 0 and source[index - 1] != '\n') and source.find('#', index + 1) != -1:
                index = source.index('#', index + 1)
            if index == 0 or source[index - 1] == '\n':
                nb.cells[count].source = set_link_in_source(source, add_link, index, text)
                break
        count += 1
    write_ipynb(nb, notebook_path)

## Convert filepath in Markdown text

In [ ]:
def get_file_md(folder_nice, folder_url, files, title_sep="##", subtitle_sep="*"):
    md = ""
    if (len(files) > 0):
        md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
    #                 print(file)

            if file.endswith(notebook_ext):
                file_url = urllib.parse.quote(file)
                file_nice = file.replace('_', ' ')
                file_nice = file_nice.replace(notebook_ext, '')
                file_nice = file_nice.replace(folder_nice, '')
                file_nice = file_nice.strip()
                file_nice = file_nice[0].capitalize() + file_nice[1:]
                try:
                    set_notebook_link(urllib.parse.unquote(f"{folder_url}/{file_url}"), f"{naas_download_url}{github_download_url}{folder_url}/{file_url}")
                except:
                    e = sys.exc_info()[0]
                md += f"{subtitle_sep} [{file_nice}]({github_url}/{folder_url}/{file_url})\n"
    return md

## Create list of all notebooks

In [ ]:
generated_list = ""

for cur in total:
    root = cur.get('root')
    md_round = ""
    directories = cur.get('directories') 
    files = cur.get('files')
    if ('.git' not in root and '.ipynb_checkpoints' not in root and '.' != root):
#         print(root)
#         print(root, files, '\n')
        folder_nice = root.replace('./', '')
        folder_url = urllib.parse.quote(folder_nice)
        if ('/' not in folder_nice):
            md_round += get_file_md(folder_nice, folder_url, files)
        elif ('/' in folder_nice):
            folder_url = urllib.parse.quote(folder_nice)
            subfolder_nice = folder_nice.split('/')[1].replace('_', ' ').replace(folder_nice, '').strip()
            md_round += get_file_md(subfolder_nice, folder_url, files, "\t###", "\t-")
    elif ('.ipynb_checkpoints' in root):
         print(root, files)
         for file in files:
            try:
                os.remove(os.path.join(root, file))
            except:
                pass
         try:
            os.rmdir(root)
         except:
            pass
#     print(md_round)
    generated_list += md_round
      

## Preview the generated list

In [ ]:
naas_drivers.markdown.display(generated_list)

In [ ]:
template = open(readme_template).read()

template = template.replace(replace_var, generated_list)

f  = open(readme, "w+")
f.write(template)
f.close()